In [3]:
# Import necessary libraries
import json
import os
import shutil
import cv2
import numpy as np
import random
from pathlib import Path
from glob import glob
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
# Load and print JSON data
with open("WLASL/start_kit/WLASL_v0.3.json", "r", encoding="utf-8") as file:
    data = json.load(file)
    print(type(data))
    print(data[:2])


<class 'list'>
[{'gloss': 'book', 'instances': [{'bbox': [385, 37, 885, 720], 'fps': 25, 'frame_end': -1, 'frame_start': 1, 'instance_id': 0, 'signer_id': 118, 'source': 'aslbrick', 'split': 'train', 'url': 'http://aslbricks.org/New/ASL-Videos/book.mp4', 'variation_id': 0, 'video_id': '69241'}, {'bbox': [190, 25, 489, 370], 'fps': 25, 'frame_end': -1, 'frame_start': 1, 'instance_id': 1, 'signer_id': 90, 'source': 'aslsignbank', 'split': 'train', 'url': 'https://aslsignbank.haskins.yale.edu/dictionary/protected_media/glossvideo/ASL/BO/BOOK-418.mp4', 'variation_id': 0, 'video_id': '65225'}, {'bbox': [262, 1, 652, 480], 'fps': 25, 'frame_end': -1, 'frame_start': 1, 'instance_id': 2, 'signer_id': 110, 'source': 'valencia-asl', 'split': 'train', 'url': 'https://www.youtube.com/watch?v=0UsjUE-TXns', 'variation_id': 0, 'video_id': '68011'}, {'bbox': [123, 19, 516, 358], 'fps': 25, 'frame_end': 60, 'frame_start': 1, 'instance_id': 3, 'signer_id': 113, 'source': 'lillybauer', 'split': 'train', 

In [ ]:
# Find and print wanted glosses
wanted_glosses = {"hello", "thank you", "yes", "no", "please", "help", "stop", "sorry", "i", "you"}

file_path = "WLASL/start_kit/WLASL_v0.3.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

found_glosses = {item["gloss"] for item in data if item["gloss"] in wanted_glosses}

if found_glosses:
    print("Found glosses:", found_glosses)
else:
    print("None of the wanted glosses were found.")


Found glosses: {'i', 'no', 'you', 'please', 'stop', 'yes', 'help', 'sorry', 'hello', 'thank you'}


In [ ]:
# Filter and save JSON data
wanted_glosses = {"hello", "thank you", "yes", "no", "please", "help", "stop", "sorry", "i", "you"}

file_path = "WLASL/start_kit/WLASL_v0.3.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

filtered_data = [entry for entry in data if entry["gloss"] in wanted_glosses]

output_file = "WLASL/start_kit/WLASL_filtered.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(filtered_data, f, indent=4)

print(f"Filtered JSON saved as {output_file}")

Filtered JSON saved as WLASL/start_kit/WLASL_filtered.json


In [ ]:
# Organize videos into folders based on glosses
json_path = "WLASL_filtered.json"
videos_path = "videos"
organized_videos_path = "organized_videos"

with open(json_path, "r") as f:
    data = json.load(f)

os.makedirs(organized_videos_path, exist_ok=True)

for item in data:
    gloss = item["gloss"]
    instances = item["instances"]
    gloss_dir = os.path.join(organized_videos_path, gloss)
    os.makedirs(gloss_dir, exist_ok=True)
    
    for instance in instances:
        video_filename = f"{instance['video_id']}.mp4"
        source_path = os.path.join(videos_path, video_filename)
        dest_path = os.path.join(gloss_dir, f"video-{video_filename}")
        
        if os.path.exists(source_path):
            shutil.move(source_path, dest_path)
            print(f"Moved: {source_path} -> {dest_path}")
        else:
            print(f"Warning: {source_path} not found.")

print("Organizing complete!")

Moved: videos/27194.mp4 -> organized_videos/help/video-27194.mp4
Moved: videos/69364.mp4 -> organized_videos/help/video-69364.mp4
Moved: videos/27206.mp4 -> organized_videos/help/video-27206.mp4
Moved: videos/27207.mp4 -> organized_videos/help/video-27207.mp4
Moved: videos/27208.mp4 -> organized_videos/help/video-27208.mp4
Moved: videos/27209.mp4 -> organized_videos/help/video-27209.mp4
Moved: videos/65890.mp4 -> organized_videos/help/video-65890.mp4
Moved: videos/27213.mp4 -> organized_videos/help/video-27213.mp4
Moved: videos/27214.mp4 -> organized_videos/help/video-27214.mp4
Moved: videos/65891.mp4 -> organized_videos/help/video-65891.mp4
Moved: videos/65889.mp4 -> organized_videos/help/video-65889.mp4
Moved: videos/27215.mp4 -> organized_videos/help/video-27215.mp4
Moved: videos/27216.mp4 -> organized_videos/help/video-27216.mp4
Moved: videos/27217.mp4 -> organized_videos/help/video-27217.mp4
Moved: videos/38482.mp4 -> organized_videos/no/video-38482.mp4
Moved: videos/69411.mp4 -> 

In [ ]:
# Process videos to a fixed size and frame count
TARGET_WIDTH = 224
TARGET_HEIGHT = 224
TARGET_FRAMES = 16  # Fixed number of frames for 3D CNN

video_dir = "organized_videos"
output_dir = "processed_videos"
os.makedirs(output_dir, exist_ok=True)

def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = total_frames / TARGET_FRAMES if total_frames >= TARGET_FRAMES else 1
    
    frames = []
    for i in range(TARGET_FRAMES):
        frame_idx = int(i * frame_rate)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (TARGET_WIDTH, TARGET_HEIGHT))
        frames.append(frame)
    
    cap.release()
    
    if len(frames) < TARGET_FRAMES:
        last_frame = frames[-1] if frames else np.zeros((TARGET_HEIGHT, TARGET_WIDTH, 3), dtype=np.uint8)
        frames += [last_frame] * (TARGET_FRAMES - len(frames))
    
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 15, (TARGET_WIDTH, TARGET_HEIGHT))
    for frame in frames:
        out.write(frame)
    out.release()

for word_sign in os.listdir(video_dir):
    sign_path = os.path.join(video_dir, word_sign)
    output_sign_path = os.path.join(output_dir, word_sign)
    os.makedirs(output_sign_path, exist_ok=True)
    
    for video_file in os.listdir(sign_path):
        if video_file.endswith(".mp4"):
            input_video_path = os.path.join(sign_path, video_file)
            output_video_path = os.path.join(output_sign_path, video_file)
            process_video(input_video_path, output_video_path)
            print(f"Processed: {video_file}")

print("Video processing complete!")


Processed: video-27180.mp4
Processed: video-70017.mp4
Processed: video-27181.mp4
Processed: video-27177.mp4
Processed: video-27171.mp4
Processed: video-27173.mp4
Processed: video-27172.mp4
Processed: video-53367.mp4
Processed: video-53366.mp4
Processed: video-53371.mp4
Processed: video-53373.mp4
Processed: video-53365.mp4
Processed: video-53370.mp4
Processed: video-53372.mp4
Processed: video-53363.mp4
Processed: video-70225.mp4
Processed: video-28557.mp4
Processed: video-28789.mp4
Processed: video-28791.mp4
Processed: video-28790.mp4
Processed: video-57656.mp4
Processed: video-57663.mp4
Processed: video-57655.mp4
Processed: video-57659.mp4
Processed: video-57653.mp4
Processed: video-57657.mp4
Processed: video-69502.mp4
Processed: video-43222.mp4
Processed: video-43218.mp4
Processed: video-43224.mp4
Processed: video-43217.mp4
Processed: video-69434.mp4
Processed: video-70220.mp4
Processed: video-43213.mp4
Processed: video-43219.mp4
Processed: video-43220.mp4
Processed: video-38541.mp4
P

In [ ]:
# Augment videos to reach target count
VIDEOS_DIR = "processed_videos"
TARGET_VIDEOS_PER_SIGN = 50
FRAME_SIZE = (224, 224)
NUM_FRAMES = 16

# Augmentations
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    A.GaussianBlur(blur_limit=(3, 7), p=0.3),
    A.Affine(rotate=(-20, 20), shear=(-10, 10), p=0.5),
    ToTensorV2()
])

def load_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, FRAME_SIZE)
        frames.append(frame)
    cap.release()
    return frames

def adjust_frames(frames):
    if len(frames) > NUM_FRAMES:
        drop_indices = sorted(random.sample(range(len(frames)), len(frames) - NUM_FRAMES))
        frames = [frame for i, frame in enumerate(frames) if i not in drop_indices]
    elif len(frames) < NUM_FRAMES:
        while len(frames) < NUM_FRAMES:
            dup_index = random.randint(0, len(frames) - 1)
            frames.insert(dup_index, frames[dup_index].copy())
    return frames

def augment_video(frames):
    augmented_frames = [transform(image=frame)['image'].permute(1, 2, 0).numpy() for frame in frames]
    return augmented_frames

def save_video(frames, save_path, fps=25):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(save_path, fourcc, fps, FRAME_SIZE)
    for frame in frames:
        out.write(frame.astype(np.uint8))
    out.release()

def process_sign_videos(sign_folder, sign_name):
    video_paths = glob(os.path.join(sign_folder, "*.mp4"))
    num_videos = len(video_paths)
    augment_needed = TARGET_VIDEOS_PER_SIGN - num_videos
    print(f"{sign_name}: {num_videos} existing videos, need {augment_needed} more.")
    
    for i in range(augment_needed):
        src_video = random.choice(video_paths)
        frames = load_video_frames(src_video)
        frames = adjust_frames(frames)
        frames = augment_video(frames)
        aug_video_path = os.path.join(sign_folder, f"aug_{i}.mp4")
        save_video(frames, aug_video_path)


sign_folders = [f for f in os.listdir(VIDEOS_DIR) if os.path.isdir(os.path.join(VIDEOS_DIR, f))]
for sign in tqdm(sign_folders):
    process_sign_videos(os.path.join(VIDEOS_DIR, sign), sign)
print("Augmentation complete!")


  0%|          | 0/10 [00:00<?, ?it/s]

hello: 7 existing videos, need 43 more.


 10%|█         | 1/10 [00:03<00:30,  3.36s/it]

sorry: 9 existing videos, need 41 more.


 20%|██        | 2/10 [00:06<00:25,  3.15s/it]

...

you: 12 existing videos, need 38 more.


 90%|█████████ | 9/10 [00:23<00:02,  2.41s/it]

yes: 16 existing videos, need 34 more.


100%|██████████| 10/10 [00:25<00:00,  2.59s/it]

Augmentation complete!


In [ ]:
# Rename processed videos with a consistent naming convention
processed_videos_dir = "processed_videos"

for subdir in os.listdir(processed_videos_dir):
    subdir_path = os.path.join(processed_videos_dir, subdir)
    if os.path.isdir(subdir_path):
        for idx, video_file in enumerate(os.listdir(subdir_path)):
            if video_file.endswith(".mp4"):
                old_path = os.path.join(subdir_path, video_file)
                new_filename = f"{subdir}-{idx + 1}.mp4"
                new_path = os.path.join(subdir_path, new_filename)
                os.rename(old_path, new_path)
                print(f"Renamed: {old_path} -> {new_path}")

print("Renaming complete!")

Renamed: processed_videos/hello/aug_9.mp4 -> processed_videos/hello/hello-1.mp4
Renamed: processed_videos/hello/aug_30.mp4 -> processed_videos/hello/hello-2.mp4
Renamed: processed_videos/hello/aug_16.mp4 -> processed_videos/hello/hello-3.mp4
Renamed: processed_videos/hello/aug_29.mp4 -> processed_videos/hello/hello-4.mp4
Renamed: processed_videos/hello/aug_18.mp4 -> processed_videos/hello/hello-5.mp4
Renamed: processed_videos/hello/aug_20.mp4 -> processed_videos/hello/hello-6.mp4
Renamed: processed_videos/hello/aug_5.mp4 -> processed_videos/hello/hello-7.mp4
Renamed: processed_videos/hello/aug_22.mp4 -> processed_videos/hello/hello-8.mp4
Renamed: processed_videos/hello/aug_23.mp4 -> processed_videos/hello/hello-9.mp4
Renamed: processed_videos/hello/aug_27.mp4 -> processed_videos/hello/hello-10.mp4
Renamed: processed_videos/hello/aug_14.mp4 -> processed_videos/hello/hello-11.mp4
Renamed: processed_videos/hello/aug_34.mp4 -> processed_videos/hello/hello-12.mp4
Renamed: processed_videos/h

In [ ]:
sk_6a32898ebb49e17e37094627ed85de9d903d6f3e4e5702bf

In [3]:
from elevenlabs import set_api_key

set_api_key("sk_6a32898ebb49e17e37094627ed85de9d903d6f3e4e5702bf")


ImportError: cannot import name 'set_api_key' from 'elevenlabs' (/home/orzl/Desktop/signspeak-hackathon/.venv/lib/python3.10/site-packages/elevenlabs/__init__.py)